In [1]:
import theano
from theano import tensor as T
import numpy as np
from load import mnist
#from foxhound.vis import grayscale_grid_vis, unit_scale
from foxhound.vis import grayscale_grid_vis
from scipy.misc import imsave

In [2]:
def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

In [3]:
def init_weights(shape):
    return theano.shared(floatX(np.random.randn(*shape) * 0.01))

In [4]:
def sgd(cost, params, lr=0.05):
    grads = T.grad(cost=cost, wrt=params)
    updates = []
    for p, g in zip(params, grads):
        updates.append([p, p - g * lr])
    return updates

In [5]:
def model(X, w_h, w_o):
    h = T.nnet.sigmoid(T.dot(X, w_h))
    pyx = T.nnet.softmax(T.dot(h, w_o))
    return pyx

In [6]:
trX, teX, trY, teY = mnist(onehot=True)

X = T.fmatrix()
Y = T.fmatrix()

In [7]:
w_h = init_weights((784, 625))
w_o = init_weights((625, 10))


In [8]:
py_x = model(X, w_h, w_o)
y_x = T.argmax(py_x, axis=1)


In [9]:
cost = T.mean(T.nnet.categorical_crossentropy(py_x, Y))
params = [w_h, w_o]
updates = sgd(cost, params)

In [10]:
train = theano.function(inputs=[X, Y], outputs=cost, updates=updates, allow_input_downcast=True)
predict = theano.function(inputs=[X], outputs=y_x, allow_input_downcast=True)


WARNING (theano.gof.compilelock): Overriding existing lock by dead process '13136' (I am process '6240')
INFO (theano.gof.compilelock): Refreshing lock C:\Users\chyam\AppData\Local\Theano\compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_61_Stepping_4_GenuineIntel-2.7.12-64\lock_dir\lock
INFO (theano.gof.compilelock): Refreshing lock C:\Users\chyam\AppData\Local\Theano\compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_61_Stepping_4_GenuineIntel-2.7.12-64\lock_dir\lock
INFO (theano.gof.compilelock): Refreshing lock C:\Users\chyam\AppData\Local\Theano\compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_61_Stepping_4_GenuineIntel-2.7.12-64\lock_dir\lock


In [11]:
for i in range(100):
    for start, end in zip(range(0, len(trX), 128), range(128, len(trX), 128)):
        cost = train(trX[start:end], trY[start:end])
    print np.mean(np.argmax(teY, axis=1) == predict(teX))


0.7024
0.8289
0.8682
0.8837
0.8891
0.8947
0.8987
0.9019
0.9046
0.9067
0.9094
0.911
0.9127
0.9136
0.9149
0.9154
0.9163
0.9167
0.9178
0.9184
0.9186
0.9192
0.9196
0.9199
0.9206
0.9211
0.9209
0.9217
0.9223
0.9227
0.9231
0.9241
0.9241
0.9245
0.9254
0.9258
0.9261
0.9266
0.9273
0.9281
0.9286
0.929
0.93
0.9304
0.9309
0.9315
0.9325
0.933
0.9335
0.9343
0.9349
0.9354
0.9358
0.9368
0.938
0.9382
0.9385
0.9391
0.9394
0.9397
0.9406
0.9415
0.9426
0.943
0.9432
0.9443
0.9451
0.9455
0.9459
0.9462
0.9464
0.9468
0.9474
0.9476
0.948
0.948
0.9484
0.949
0.9495
0.95
0.9506
0.9508
0.9514
0.9517
0.9521
0.9526
0.9529
0.9533
0.9535
0.9542
0.9544
0.955
0.9553
0.9553
0.9556
0.956
0.956
0.9564
0.9569
0.9574
